# Análise de Indicadores Socioeconômicos e Votação (Brasil 2022)
Este notebook carrega e combina:
1. **Indicadores socioeconômicos por Unidade da Federação (UF) em 2022**
2. **Resultado de votos nominais válidos do 1º turno das eleições de 2022**, agregados por UF e **agrupados por ideologia partidária**.

O resultado final é salvo em um CSV (`uf_votos_ideologia_socioeco_2022.csv`).

In [1]:
import pandas as pd
from pathlib import Path

## Definição dos caminhos dos arquivos de entrada
Ajuste conforme seu diretório:

In [2]:
BASE = Path('.')  # ajuste se necessário

# Arquivos de entrada
path_indicadores = BASE / '/home/fbotero/Documents/POS/data_lake/raw/csv/indicadores_uf_2022.csv'
path_gini         = BASE / '/home/fbotero/Documents/POS/data_lake/raw/csv/gini_uf_2022.csv'
path_analf        = BASE / '/home/fbotero/Documents/POS/data_lake/raw/csv/analfabetismo_uf_2022.csv'
path_votos        = BASE / '/home/fbotero/Documents/POS/data_lake/raw/csv/votacao_partido_munzona_2022_BR.csv'

# Saída
out_csv           = BASE / '/home/fbotero/Documents/POS/data_lake/processed/uf_votos_ideologia_socioeco_2022.csv'

## 1. Carregar e padronizar indicadores socioeconômicos

In [3]:
ind = pd.read_csv(path_indicadores)

# Renomear e ajustar colunas
ind.rename(columns={'PIB_R$mi': 'PIB_milhoes',
                    'PIB_per_capita_R$': 'PIB_per_capita',
                    'Esperanca_vida': 'Esperanca_vida_anos',
                    'Mort_infantil_per_mil': 'Mort_infantil_pmil',
                    'Analfabetismo_%': 'Analfabetismo_perc',
                    'IPCA_2022_%': 'IPCA_2022_perc'}, inplace=True)

# Completar Gini, caso não exista
if 'Gini' not in ind.columns:
    gini = pd.read_csv(path_gini)
    ind = ind.merge(gini[['UF', '2022']].rename(columns={'2022': 'Gini'}), on='UF', how='left')

# Completar Analfabetismo, caso ausente
if 'Analfabetismo_perc' not in ind.columns:
    analf = pd.read_csv(path_analf)
    ind = ind.merge(analf[['UF', '2022']].rename(columns={'2022': 'Analfabetismo_perc'}), on='UF', how='left')

ind.head()

,UF,PIB_milhoes,PIB_per_capita,IDHM_2021,Esperanca_vida_anos,Mort_infantil_pmil,Gini,Analfabetismo_perc
0,AC,23676,28525,"0,71",75.3,15.5,0.52,12.13
1,AL,76066,24322,"0,684",73.3,13.8,0.50,17.66
2,AM,145140,36827,"0,7",73.0,16.0,0.51,6.94
3,AP,23614,32194,"0,688",75.0,18.1,0.53,6.47
4,BA,402647,28483,"0,691",74.5,14.0,0.51,12.60


## 2. Carregar votos nominais e atribuir ideologia partidária

In [4]:
votes = pd.read_csv(path_votos, sep=';', encoding='latin-1',
                     usecols=['NR_TURNO', 'SG_UF', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS_VALIDOS'])

# Filtra 1º turno e votos > 0
votes = votes[(votes['NR_TURNO'] == 1) & (votes['QT_VOTOS_NOMINAIS_VALIDOS'] > 0)]

# Mapeamento partido -> ideologia
ideology_map = {
    'PT':'esquerda','PCB':'esquerda','UP':'esquerda','PSTU':'esquerda',
    'PDT':'centro-esquerda','PSB':'centro-esquerda','REDE':'centro-esquerda','PV':'centro-esquerda',
    'MDB':'centro','PSD':'centro','PODE':'centro','CIDADANIA':'centro','AVANTE':'centro',
    'UNIÃO':'centro-direita','PSDB':'centro-direita','PP':'centro-direita','NOVO':'centro-direita',
    'SOLIDARIEDADE':'centro-direita',
    'PL':'direita','PTB':'direita','DC':'direita','PATRIOTA':'direita','PRTB':'direita'
}

votes['ideologia'] = votes['SG_PARTIDO'].map(ideology_map).fillna('outros')
votes.head()

,NR_TURNO,SG_UF,SG_PARTIDO,QT_VOTOS_NOMINAIS_VALIDOS,ideologia
33,1,CE,MDB,1761,centro
34,1,SC,MDB,209,centro
35,1,RS,MDB,81,centro
36,1,SC,MDB,115,centro
37,1,PB,MDB,60,centro


## 3. Agregar votos por UF e ideologia

In [5]:
agg = (votes.groupby(['SG_UF','ideologia'])['QT_VOTOS_NOMINAIS_VALIDOS']
         .sum().unstack(fill_value=0).reset_index().rename(columns={'SG_UF':'UF'}))

# Calcula participação percentual
ideology_cols = [c for c in agg.columns if c != 'UF']
agg['total_votos'] = agg[ideology_cols].sum(axis=1)
for col in ideology_cols:
    agg[col + '_perc'] = (agg[col] / agg['total_votos']) * 100

agg.head()

ideologia,UF,centro,centro-direita,centro-esquerda,direita,esquerda,total_votos,centro_perc,centro-direita_perc,centro-esquerda_perc,direita_perc,esquerda_perc
0,AC,20122,3269,12314,275905,129307,440917,4.563671,0.741409,2.792816,62.575269,29.326835
1,AL,67411,13133,43542,624674,975457,1724217,3.909659,0.761679,2.525320,36.229431,56.573912
2,AM,87060,22214,44527,881838,1021037,2056676,4.233044,1.080092,2.164998,42.876856,49.645010
3,AP,27497,4191,14670,188042,197761,432161,6.362675,0.969777,3.394568,43.512024,45.760955
4,BA,197305,76651,217224,2053250,5878578,8423008,2.342453,0.910019,2.578936,24.376683,69.791908


## 4. Combinar indicadores e votação

In [6]:
combined = ind.merge(agg, on='UF', how='left')
combined.head()

,UF,PIB_milhoes,PIB_per_capita,IDHM_2021,Esperanca_vida_anos,Mort_infantil_pmil,Gini,Analfabetismo_perc,centro,centro-direita,centro-esquerda,direita,esquerda,total_votos,centro_perc,centro-direita_perc,centro-esquerda_perc,direita_perc,esquerda_perc
0,AC,23676,28525,"0,71",75.3,15.5,0.52,12.13,20122,3269,12314,275905,129307,440917,4.563671,0.741409,2.792816,62.575269,29.326835
1,AL,76066,24322,"0,684",73.3,13.8,0.50,17.66,67411,13133,43542,624674,975457,1724217,3.909659,0.761679,2.525320,36.229431,56.573912
2,AM,145140,36827,"0,7",73.0,16.0,0.51,6.94,87060,22214,44527,881838,1021037,2056676,4.233044,1.080092,2.164998,42.876856,49.645010
3,AP,23614,32194,"0,688",75.0,18.1,0.53,6.47,27497,4191,14670,188042,197761,432161,6.362675,0.969777,3.394568,43.512024,45.760955
4,BA,402647,28483,"0,691",74.5,14.0,0.51,12.60,197305,76651,217224,2053250,5878578,8423008,2.342453,0.910019,2.578936,24.376683,69.791908


## 5. Salvar conjunto combinado em CSV

In [7]:
combined.to_csv(out_csv, index=False)
print(f'Arquivo salvo em: {out_csv.resolve()}')

Arquivo salvo em: /home/fbotero/Documents/POS/data_lake/processed/uf_votos_ideologia_socioeco_2022.csv


### Pronto!
Agora você tem um dataset unificado `uf_votos_ideologia_socioeco_2022.csv` com indicadores socioeconômicos e votação por bloco ideológico. A partir daqui, você pode seguir para análises estatísticas, visualizações ou modelos de ML.